The below cell contains all the constants used in rest of the codes

# Constants

In [1]:
"""
Created on Thu Feb 20 08:22:30 2020

@author: Saba
"""
BACK_BONE = 'efficientnetb5'
BATCH_SIZE = 2
Defined_CLASSES = [1]
LR = 0.0001
EPOCHS = 200

NO_TIME_STEPS = 4 #Number of Time_steps
SPLIT_MODE = [0.65, 0.8] #60% Train, (80-60)=20% Val, (100-80)=20% Test

ENCODER_WEIGHT = 'imagenet'
ENCODER_FREEZE = True
MODEL_INPUT_SHAPE = (None,None,3)

In [2]:
import keras
OPTIMIZER = keras.optimizers.Adam(LR)

Using TensorFlow backend.


In [3]:
HOME_DIR = "/home/ubuntu/"

PATH_TO_SAVE_MODEL = HOME_DIR +'/Trained_model/'

MODEL_NAME = "_POST_LSTM" # _MULTIPLICATION, _CONCAT, _PRE_LSTM, _POST_LSTM, _Nine_W_1_1_CONV, _Nine_WO_1_1_CONV, _PRE_LSTM_UNET_MUL, _PRE_LSTM_UNET_CONCAT

if ENCODER_FREEZE :
    f = 'True'
else:
    f = 'False'
    
if ENCODER_WEIGHT is None:
    w = 'None'
else:
    w = ENCODER_WEIGHT
MODEL_DESC = '{'+BACK_BONE+'}_batch_{'+str(BATCH_SIZE)+'}_epoch_{'+str(EPOCHS)+'}_time_steps_{'+str(NO_TIME_STEPS)+'}_encoder_freeze_{'+f+'}_encoder_weight_{'+w+'}{'+MODEL_NAME+'}'+'_4_3_2_with_shared_weight'
print(MODEL_DESC)
PATH_TO_INPUT = HOME_DIR + "Dataset/fields_with_masks/"

In [4]:
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.15.2
Eager execution: False


In [5]:
import matplotlib.pyplot as plt
from io import BytesIO
import matplotlib.image as mpimg
import numpy as np
import cv2
import re

In [6]:
""" This function used to visualize the input, predicted mask and ground truth"""
def visualize(save_status, save_path,flight_code,**images ):
    
    n = len(images)
    plt.figure(figsize=(40,40))
    for i, (name , image) in enumerate(images.items()):
        plt.subplot(1, n, i+1)
        plt.xticks([])
        plt.yticks([])
        # plt.title(str(i+1))
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
        
    
    if save_status : 
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        plt.savefig(save_path+flight_code+'.jpg')    
    plt.show()
   

In [7]:
def denormalize(x):
    x_max = np.max(x)
    x_min = np.min(x)
    x = (x-x_min) / (x_max - x_min)
    x = x.clip(0,1)
    return x

In [8]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key,reverse=True)   

# Creating Dataset

In [9]:
class Dataset:
    
    CLASSES = Defined_CLASSES
    def __init__(
            self,
            field_dir,
            status, #train,test,validation
            classes = None,
            augmentation = None,
            preprocessing = None
            ):
        """ make a list of image directions  """
        files = ['bounday_mask.png','nutrient_mask.png']
        self.field_dirs = [ids for ids in sorted(os.listdir(field_dir))]
        self.field_fps = [os.path.join(field_dir ,field_id) for field_id in self.field_dirs if len(os.listdir(os.path.join(field_dir ,field_id))) >= NO_TIME_STEPS ]
        
        train, validate, test = np.split(self.field_fps, [int(SPLIT_MODE[0]*len(self.field_fps)), int(SPLIT_MODE[1]*len(self.field_fps))])
        print(len(train))
        print(len(validate))
        print(len(test))
        if status == 'train':
            self.field_fps = train
        elif status == 'validate':
            self.field_fps = validate
        elif status == 'test':
            self.field_fps = test
        
        self.field_codes            = []
        self.field_number            = []
        self.field_images            = []
        self.field_nutrient_mask     = []
        self.field_boundary_mask     = []

        for field_path in self.field_fps:
            flight_dirs = sorted_alphanumeric(os.listdir(field_path))
            codes  = []
            number = []
            images = []
            nutrient_mask = []
            boundary_mask = []

            for j in flight_dirs:
                flight_number, flight_code = j.split('_')
                codes.append(flight_code)
                number.append(flight_number)
                images.append(os.path.join(field_path,j,flight_code+'.png'))
                boundary_mask.append(os.path.join(field_path,j,files[0]))
                nutrient_mask.append(os.path.join(field_path,j,files[1]))


            self.field_codes.append(codes)
            self.field_number.append(number)
            self.field_images.append(images)
            self.field_nutrient_mask.append(nutrient_mask)
            self.field_boundary_mask.append(boundary_mask)

        
        """ set the class values and assign a augmentation and preprocessing method"""
        self.class_values = classes
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        
    def __getitem__(self,i):
         
        """ read images"""
        image1 = cv2.imread(self.field_images[i][0])
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        w, h,_ = image1.shape
        
        
        image2 = cv2.imread(self.field_images[i][1])
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
        image2 = cv2.resize(image2, (h,w) , interpolation = cv2.INTER_AREA)
        
        
        image3 = cv2.imread(self.field_images[i][2])
        image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
        image3 = cv2.resize(image3, (h,w), interpolation = cv2.INTER_AREA)
        
        image4 = cv2.imread(self.field_images[i][3])
        image4 = cv2.cvtColor(image4, cv2.COLOR_BGR2RGB)
        image4 = cv2.resize(image4, (h,w) , interpolation = cv2.INTER_AREA)
        
        
#         image5 = cv2.imread(self.field_images[i][4])
#         image5 = cv2.cvtColor(image5, cv2.COLOR_BGR2RGB)
#         image5 = cv2.resize(image5, (h,w), interpolation = cv2.INTER_AREA)
        
        
        
        boundary_mask = cv2.imread(self.field_boundary_mask[i][0])
        boundary_mask = cv2.cvtColor(boundary_mask, cv2.COLOR_BGR2GRAY)
        
        
        nutrient_mask = cv2.imread(self.field_nutrient_mask[i][0])
        nutrient_mask = cv2.cvtColor(nutrient_mask, cv2.COLOR_BGR2GRAY) 
        
        boundary_mask = denormalize(boundary_mask)
        nutrient_mask = denormalize(nutrient_mask)
        
        """ extract certain classes from mask 1 """
        masks = [(boundary_mask == v) for v in self.class_values]
        boundary_mask = np.stack(masks, axis = -1).astype('float')
        
        boundary_mask_for_slic = np.stack(masks, axis = 0).astype('float')
        
        masks = [(nutrient_mask == v) for v in self.class_values]
        nutrient_mask = np.stack(masks, axis = -1).astype('float')
        
        data1 = {"image": image1,"image2": image2, "image3": image3,"image4": image4, "mask": nutrient_mask }
        """ apply augmentation """
        if self.augmentation:
            sample = self.augmentation(**data1)
            image1 , nutrient_mask, image2, image3, image4 = sample['image'], sample['mask'], sample['image2'], sample['image3'], sample['image4'] 
         
        data2 = {"image": image1,"image2": image2, "image3": image3,"image4": image4, "mask": nutrient_mask }
        """ apply preprocessing """
        if self.preprocessing:
            sample = self.preprocessing(**data2)
            image1 , nutrient_mask,image2,image3,image4 = sample['image'], sample['mask'], sample['image2'],sample['image3'], sample['image4']
        
        return image1,image2, image3,image4,nutrient_mask, boundary_mask,self.field_codes[i][0] 
    
    def __len__(self):
        return len(self.field_images)
    

## Dataloader

In [10]:
class Dataloader(keras.utils.Sequence):
    
    def __init__(
            self,
            dataset,
            batch_size = 1,
            shuffle = False
            ):
        
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))
        
        self.on_epoch_end()
        
    def __getitem__(self, i):
        
        start = i * self.batch_size
        stop = (i+1) * self.batch_size
        data1 = []
        data2 = []
        data3 = []
        masks = []
        data_concat = []
        data_stack = []
        masks_stack = []
        for j in range(start, stop):
            im1,im2,im3,im4,mask,_,_ = self.dataset[j]
            data1.append(im1)
            data2.append(im2)
            data3.append(im3)
            masks.append(mask)
            
            #im1,im2,im3,mask,_,_ = self.dataset[j]
            image_concat = np.concatenate((im1,im2,im3),axis =2)
            data_concat.append((image_concat,mask))
            
            image_stack = np.stack((im4,im3,im2),axis=0)
            #m_stack = np.stack((mask,mask,mask),axis=0)
            m_stack = np.expand_dims(mask,axis = 0)
            data_stack.append(image_stack)
            masks_stack.append(m_stack)
        
        if MODEL_NAME == '_Nine_W_1_1_CONV' or MODEL_NAME == '_Nine_WO_1_1_CONV':
            batch = [np.stack(samples, axis = 0) for samples in zip(*data_concat)]
            return batch
        elif MODEL_NAME == '_PRE_LSTM' or MODEL_NAME == '_POST_LSTM':
            #batch = [np.stack(samples, axis = 0) for samples in zip(*data_stack)]
            return [np.array(data_stack)],[np.array(masks_stack),np.array(masks_stack),np.array(masks_stack)]
        else:
            return [np.array(data1),np.array(data2),np.array(data3)],[np.array(masks),np.array(masks),np.array(masks)]
    
    def __len__(self):
        """  the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)            

# Do Data Augmentation

In [11]:
import albumentations as A


def round_clip_0_1(x, **kwargs):
    return x.round

def training_augmentation():
        train_transform = [
            A.HorizontalFlip(p = 0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(scale_limit = 0.5, rotate_limit=0, shift_limit = 0.1, p = 1, border_mode=0),
            A.PadIfNeeded(min_height = 512, min_width = 512, always_apply= True, border_mode=0),
            #A.RandomCrop(height=512, width=512, always_apply = True),
#             A.CropNonEmptyMaskIfExists(height=512, width=512,p=1),
            #A.RandomResizedCrop(height, width, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=1.0)
            
            A.IAAAdditiveGaussianNoise(p=0.2),
            A.IAAPerspective(p=0.5),
            
            A.OneOf(
                [
                    A.CropNonEmptyMaskIfExists(height=512, width=512,p=0.5),
                    A.Resize(height=512, width=512, interpolation=1, p=0.5),
                ],
                p = 1,
            ),
            A.OneOf(
               [
                   A.CLAHE(p=1),
                   A.RandomBrightness(p=1),
                   A.RandomGamma(p=1),
               ],
               p=0.9,
            ),

            A.OneOf(
               [
                   A.IAASharpen(p=1),
                   A.Blur(blur_limit=3, p=1),
                   A.MotionBlur(blur_limit=3, p=1),
               ],
               p=0.9,
            ),

            A.OneOf(
               [
                   A.RandomContrast(p=1),
                   A.HueSaturationValue(p=1),
               ],
               p=0.9,
           ),
            # A.Lambda(mask = round_clip_0_1)
            #A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, always_apply=False, p=0.5)
        ]
    
        return A.Compose(train_transform,
            additional_targets={"image2" : "image", "image3":"image","image4" : "image"})

def validation_augmentation():
    test_transform = [
        A.PadIfNeeded(min_height = 512, min_width = 512, always_apply= True, border_mode=0),
#         A.CropNonEmptyMaskIfExists(height=512, width=512, always_apply=True)
        A.OneOf(
                [
                    A.CropNonEmptyMaskIfExists(height=512, width=512,p=0.5),
                    A.Resize(height=512, width=512, interpolation=1, p=0.5),
                ],
                p = 1,
            ),
        #A.RandomCrop(height=512, width=512, always_apply = True)
        #A.Resize(height=1024, width=1024, interpolation=1, always_apply=True, p=1)
        ]
    return A.Compose(test_transform,additional_targets={"image2" : "image", "image3":"image","image4" : "image"})

def get_preprocessing(preprocessing_fn):
    
    _transform = [
        A.Lambda(image = preprocessing_fn),
        ]
    return A.Compose(_transform,additional_targets={"image2" : "image", "image3":"image","image4" : "image"})

# Model Settings

In [12]:
import segmentation_models as sm
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, Concatenate, Multiply
from keras.layers import BatchNormalization, ConvLSTM2D, Conv3D, Lambda, Reshape
import keras.backend as K
#from tensorflow import slice

""" define preprocess method condisering back_bone"""
preprocess_input = sm.get_preprocessing(BACK_BONE)
""" model parameters"""
n_classes = 1 if len(Defined_CLASSES)== 1 else (len(Defined_CLASSES)+1)
activation = 'sigmoid' if n_classes == 1 else 'softmax'

  

Segmentation Models: using `keras` framework.


In [13]:
"""creating a model"""

class MyModels():
    def _Nine_WO_1_1_CONV(self):
        final_model = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape=(None,None,9), encoder_weights=None,encoder_freeze = ENCODER_FREEZE)
        return final_model
    def _Nine_W_1_1_CONV(self):
        model = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        
        model.layers.pop(0)
        newInputs = Input(shape=(None,None,9))
        x = Conv2D(3, kernel_size=(1,1))(newInputs)
        newOutputs = model(x)
        final_model = Model(newInputs, newOutputs)
        
        return final_model
        
    def _MULTIPLICATION(self):
        model_1 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        model_2 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        model_3 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        
        input_1 = Input(shape=(512,512,3), name='input_1')
        input_2 = Input(shape=(512,512,3), name='input_2')
        input_3 = Input(shape=(512,512,3), name='input_3')

        x3 = Model(model_1.input, model_1.output, name='model_3')(input_3)
        p = Concatenate(axis=3)([x3,x3,x3])

        input_2_new =Multiply()([input_2, p])

        x2 = Model(model_2.input, model_2.output, name='model_2')(input_2_new)
        p = Concatenate(axis=3)([x2,x2,x2])

        input_1_new =Multiply()([input_1, p])

        x1 = Model(model_3.input, model_3.output , name='model_1')(input_1_new)

        final_model = Model(inputs =[input_1,input_2,input_3],outputs=[x3,x2,x1])

        return final_model
    
    def _CONCAT(self):
        model_1 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        model_2 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        model_3 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        
        input_1 = Input(shape=(512,512,3), name='input_1')
        input_2 = Input(shape=(512,512,3), name='input_2')
        input_3 = Input(shape=(512,512,3), name='input_3')

        x3 = Model(model_1.input, model_1.output, name='model_3')(input_3)
        
        input_2_new =Concatenate(axis=3)([x3, input_2])
        x2 = Model(model_2.input, model_2.output, name='model_2')(input_2_new)

        input_1_new =Concatenate(axis=3)([x2, input_1])
        x1 = Model(model_3.input, model_3.output , name='model_1')(input_1_new)

        final_model = Model(inputs =[input_1,input_2,input_3],outputs=[x3,x2,x1])

        return final_model
    def _PRE_LSTM(self):
#         model_1 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
#         model_2 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
#         model_3 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        
        input_1 = Input(shape=(3,512,512,3), name='input')

        
        #new_input = stack((input_3,input_2,input_1), axis=1)

        x = ConvLSTM2D(filters=40, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(input_1)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=40, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=40, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=40, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = Conv3D( filters=1, kernel_size=(3, 3, 3),data_format='channels_last', activation="sigmoid", padding="same", name = 'output')(x)
#         x3 = Model(model_1.input, model_1.output, name='model_3')(input_3)
#         x2 = Model(model_2.input, model_2.output, name='model_2')(input_2_new)
#         p = Concatenate(axis=3)([x2,x2,x2])

#         input_1_new =Multiply()([input_1, p])

#         x1 = Model(model_3.input, model_3.output , name='model_1')(input_1_new)

        final_model = Model(input_1,x, name = 'hello')

        return final_model
    
    def _POST_LSTM(self):
        model_1 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        #model_2 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        #model_3 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)
        #model_4 = sm.Unet(BACK_BONE, classes = n_classes, activation = activation,input_shape = MODEL_INPUT_SHAPE, encoder_weights='imagenet',encoder_freeze = ENCODER_FREEZE)

#         input_1 = Input(shape=(512,512,3), name='input_1')
#         input_2 = Input(shape=(512,512,3), name='input_2')
#         input_3 = Input(shape=(512,512,3), name='input_3')
        combined_input = Input(shape=(3,512,512,3), name='input')
        #input_5 = Lambda( lambda x: tf.slice(x, (0,0, 0, 0,0), (-1,1,-1, -1, -1)))(combined_input)
        input_4 = Lambda( lambda x: tf.slice(x, (0,0, 0, 0,0), (-1,1,-1, -1, -1)))(combined_input)
        input_3 = Lambda( lambda x: tf.slice(x, (0,1, 0, 0,0), (-1,1,-1, -1, -1)))(combined_input)
        input_2 = Lambda( lambda x: tf.slice(x, (0,2, 0, 0,0), (-1,1,-1, -1, -1)))(combined_input)
        
        input_2 = Reshape((512,512,3))(input_2)
        input_3 = Reshape((512,512,3))(input_3)
        input_4 = Reshape((512,512,3))(input_4)
        #input_5 = Reshape((512,512,3))(input_5)

        #x5 = Model(model_1.input, model_1.output)(input_5)
        x4 = Model(model_1.input, model_1.output)(input_4)

        x3 = Model(model_1.input, model_1.output)(input_3)
        
        x2 = Model(model_1.input, model_1.output)(input_2)
        
        new_input = Lambda(lambda x: K.stack([x[0], x[1],x[2]], axis=1))([x4, x3,x2])
        
        x = ConvLSTM2D(filters=4, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(new_input)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=4, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=4, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = ConvLSTM2D(filters=4, kernel_size=(3, 3),data_format='channels_last', padding="same", return_sequences=True)(x)
        x = BatchNormalization()(x)
        x = Conv3D( filters=1, kernel_size=(3, 3, 3),data_format='channels_last', activation="sigmoid", padding="same", name = 'output')(x)

        #x5 = Lambda( lambda x: tf.slice(x, (0,0, 0, 0,0), (-1,1,-1, -1, -1)), name= 'model_4')(x)
        x4 = Lambda( lambda x: tf.slice(x, (0,0, 0, 0,0), (-1,1,-1, -1, -1)), name= 'output_4')(x)
        x3 = Lambda( lambda x: tf.slice(x, (0,1, 0, 0,0), (-1,1,-1, -1, -1)), name = 'output_3')(x)
        x2 = Lambda( lambda x: tf.slice(x, (0,2, 0, 0,0), (-1,1,-1, -1, -1)), name='output_2')(x)


        final_model = Model(inputs=combined_input,outputs=[x4,x3,x2])

        return final_model
    

In [14]:
my_cls = MyModels()
model = getattr(my_cls, MODEL_NAME)()
model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 3, 512, 512,  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1, 512, 512,  0           input[0][0]                      
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1, 512, 512,  0           input[0][0]                      
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1, 512, 512,  0           input[0][0]    

In [15]:
""" defining loss function and metrics"""
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.1,0.3,0.7,0.9])) 
#focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
#total_loss = dice_loss + (1 * focal_loss)
total_loss = sm.losses.binary_focal_dice_loss
losses = {
    #"model_4": total_loss,
    "output_4": total_loss,
    "output_3": total_loss,
    "output_2": total_loss,
}
metrics = [sm.metrics.IOUScore(threshold =0.5), sm.metrics.FScore(threshold = 0.5), sm.metrics.Precision(threshold = 0.5),sm.metrics.Recall(threshold = 0.5) ]

""" compiling the network """
model.compile(OPTIMIZER, losses, metrics)

In [16]:
""" loading traind and validation dataset """
train_dataset = Dataset(PATH_TO_INPUT,
                        status = 'train',
                           classes = Defined_CLASSES,
                           augmentation = training_augmentation(),
                           preprocessing = get_preprocessing(preprocess_input),
                           )

valid_dataset = Dataset(PATH_TO_INPUT,
                        status = 'validate',
                           classes = Defined_CLASSES,
                           augmentation = training_augmentation(),
                           preprocessing = get_preprocessing(preprocess_input),
                           )

train_dataloader = Dataloader(train_dataset, batch_size = BATCH_SIZE, shuffle = True )
valid_dataloader = Dataloader(valid_dataset, batch_size = 1, shuffle = False )

236
55
73
236
55
73


## Train the MODEL

In [ ]:
""" define callbacks  for learning rate scheduling and best checkpoints """

callbacks = [
    keras.callbacks.ModelCheckpoint(PATH_TO_SAVE_MODEL+MODEL_DESC+'.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    ]

history = model.fit_generator(
    train_dataloader,
    steps_per_epoch = len(train_dataloader),
    epochs = EPOCHS,
    callbacks = callbacks,
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader),
    )


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
 79/118 [===================>..........] - ETA: 2:47 - loss: 3.2642 - output_4_loss: 1.0599 - output_3_loss: 1.0976 - output_2_loss: 1.1067 - output_4_iou_score: 0.0572 - output_4_f1-score: 0.0986 - output_4_precision: 0.0666 - output_4_recall: 0.6954 - output_3_iou_score: 0.0550 - output_3_f1-score: 0.0955 - output_3_precision: 0.0663 - output_3_recall: 0.6465 - output_2_iou_score: 0.0459 - output_2_f1-score: 0.0810 - output_2_precision: 0.0559 - output_2_recall: 0.6036

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


118/118 [==============================] - 406s 3s/step - loss: 3.2027 - output_4_loss: 1.0553 - output_3_loss: 1.0701 - output_2_loss: 1.0773 - output_4_iou_score: 0.0661 - output_4_f1-score: 0.1107 - output_4_precision: 0.0751 - output_4_recall: 0.7403 - output_3_iou_score: 0.0611 - output_3_f1-score: 0.1045 - output_3_precision: 0.0759 - output_3_recall: 0.6496 - output_2_iou_score: 0.0455 - output_2_f1-score: 0.0806 - output_2_precision: 0.0620 - output_2_recall: 0.5606 - val_loss: 3.3289 - val_output_4_loss: 1.1087 - val_output_3_loss: 1.1017 - val_output_2_loss: 1.1185 - val_output_4_iou_score: 0.0471 - val_output_4_f1-score: 0.0761 - val_output_4_precision: 0.0484 - val_output_4_recall: 0.9589 - val_output_3_iou_score: 0.0461 - val_output_3_f1-score: 0.0749 - val_output_3_precision: 0.0496 - val_output_3_recall: 0.9107 - val_output_2_iou_score: 0.0185 - val_output_2_f1-score: 0.0339 - val_output_2_precision: 0.0244 - val_output_2_recall: 0.7236
Epoch 2/200
118/118 [=============

118/118 [==============================] - 107s 905ms/step - loss: 2.7415 - output_4_loss: 0.9247 - output_3_loss: 0.9084 - output_2_loss: 0.9084 - output_4_iou_score: 0.2220 - output_4_f1-score: 0.2702 - output_4_precision: 0.4240 - output_4_recall: 0.4852 - output_3_iou_score: 0.2068 - output_3_f1-score: 0.2612 - output_3_precision: 0.3601 - output_3_recall: 0.5353 - output_2_iou_score: 0.2033 - output_2_f1-score: 0.2612 - output_2_precision: 0.3362 - output_2_recall: 0.5670 - val_loss: 2.8799 - val_output_4_loss: 0.9659 - val_output_3_loss: 0.9558 - val_output_2_loss: 0.9583 - val_output_4_iou_score: 0.4935 - val_output_4_f1-score: 0.5106 - val_output_4_precision: 0.6919 - val_output_4_recall: 0.6714 - val_output_3_iou_score: 0.4382 - val_output_3_f1-score: 0.4601 - val_output_3_precision: 0.5859 - val_output_3_recall: 0.6983 - val_output_2_iou_score: 0.4286 - val_output_2_f1-score: 0.4529 - val_output_2_precision: 0.5601 - val_output_2_recall: 0.7223
Epoch 10/200
118/118 [=========

118/118 [==============================] - 108s 916ms/step - loss: 2.6666 - output_4_loss: 0.9063 - output_3_loss: 0.8791 - output_2_loss: 0.8812 - output_4_iou_score: 0.2357 - output_4_f1-score: 0.2914 - output_4_precision: 0.4108 - output_4_recall: 0.5206 - output_3_iou_score: 0.2382 - output_3_f1-score: 0.3033 - output_3_precision: 0.3712 - output_3_recall: 0.6059 - output_2_iou_score: 0.2323 - output_2_f1-score: 0.2973 - output_2_precision: 0.3559 - output_2_recall: 0.6100 - val_loss: 2.8291 - val_output_4_loss: 0.9530 - val_output_3_loss: 0.9377 - val_output_2_loss: 0.9385 - val_output_4_iou_score: 0.5552 - val_output_4_f1-score: 0.5715 - val_output_4_precision: 0.7929 - val_output_4_recall: 0.6458 - val_output_3_iou_score: 0.5373 - val_output_3_f1-score: 0.5608 - val_output_3_precision: 0.7246 - val_output_3_recall: 0.6801 - val_output_2_iou_score: 0.5401 - val_output_2_f1-score: 0.5630 - val_output_2_precision: 0.7247 - val_output_2_recall: 0.6765
Epoch 18/200
118/118 [=========

...

118/118 [==============================] - 108s 917ms/step - loss: 2.5342 - output_4_loss: 0.8658 - output_3_loss: 0.8341 - output_2_loss: 0.8343 - output_4_iou_score: 0.2338 - output_4_f1-score: 0.2974 - output_4_precision: 0.3864 - output_4_recall: 0.5764 - output_3_iou_score: 0.2688 - output_3_f1-score: 0.3413 - output_3_precision: 0.4181 - output_3_recall: 0.6337 - output_2_iou_score: 0.2713 - output_2_f1-score: 0.3434 - output_2_precision: 0.4221 - output_2_recall: 0.6316 - val_loss: 2.7338 - val_output_4_loss: 0.9275 - val_output_3_loss: 0.9033 - val_output_2_loss: 0.9031 - val_output_4_iou_score: 0.5843 - val_output_4_f1-score: 0.6068 - val_output_4_precision: 0.8548 - val_output_4_recall: 0.6651 - val_output_3_iou_score: 0.5715 - val_output_3_f1-score: 0.6033 - val_output_3_precision: 0.7947 - val_output_3_recall: 0.7079 - val_output_2_iou_score: 0.5746 - val_output_2_f1-score: 0.6061 - val_output_2_precision: 0.8014 - val_output_2_recall: 0.7083
Epoch 162/200
118/118 [========

118/118 [==============================] - 108s 912ms/step - loss: 2.5793 - output_4_loss: 0.8780 - output_3_loss: 0.8506 - output_2_loss: 0.8506 - output_4_iou_score: 0.2318 - output_4_f1-score: 0.2922 - output_4_precision: 0.3843 - output_4_recall: 0.5656 - output_3_iou_score: 0.2571 - output_3_f1-score: 0.3253 - output_3_precision: 0.4048 - output_3_recall: 0.6155 - output_2_iou_score: 0.2607 - output_2_f1-score: 0.3284 - output_2_precision: 0.4087 - output_2_recall: 0.6133 - val_loss: 2.7613 - val_output_4_loss: 0.9327 - val_output_3_loss: 0.9144 - val_output_2_loss: 0.9142 - val_output_4_iou_score: 0.5117 - val_output_4_f1-score: 0.5332 - val_output_4_precision: 0.7897 - val_output_4_recall: 0.6317 - val_output_3_iou_score: 0.5202 - val_output_3_f1-score: 0.5504 - val_output_3_precision: 0.7497 - val_output_3_recall: 0.6729 - val_output_2_iou_score: 0.5219 - val_output_2_f1-score: 0.5518 - val_output_2_precision: 0.7594 - val_output_2_recall: 0.6708
Epoch 170/200
118/118 [========

118/118 [==============================] - 107s 906ms/step - loss: 2.5663 - output_4_loss: 0.8747 - output_3_loss: 0.8456 - output_2_loss: 0.8460 - output_4_iou_score: 0.2420 - output_4_f1-score: 0.3050 - output_4_precision: 0.3757 - output_4_recall: 0.5883 - output_3_iou_score: 0.2669 - output_3_f1-score: 0.3382 - output_3_precision: 0.3917 - output_3_recall: 0.6400 - output_2_iou_score: 0.2685 - output_2_f1-score: 0.3394 - output_2_precision: 0.3956 - output_2_recall: 0.6385 - val_loss: 2.7268 - val_output_4_loss: 0.9251 - val_output_3_loss: 0.9007 - val_output_2_loss: 0.9010 - val_output_4_iou_score: 0.5699 - val_output_4_f1-score: 0.5914 - val_output_4_precision: 0.8192 - val_output_4_recall: 0.6587 - val_output_3_iou_score: 0.5546 - val_output_3_f1-score: 0.5872 - val_output_3_precision: 0.7511 - val_output_3_recall: 0.7092 - val_output_2_iou_score: 0.5578 - val_output_2_f1-score: 0.5898 - val_output_2_precision: 0.7521 - val_output_2_recall: 0.7086
Epoch 178/200
118/118 [========

118/118 [==============================] - 105s 894ms/step - loss: 2.5743 - output_4_loss: 0.8755 - output_3_loss: 0.8495 - output_2_loss: 0.8493 - output_4_iou_score: 0.2589 - output_4_f1-score: 0.3198 - output_4_precision: 0.3888 - output_4_recall: 0.6041 - output_3_iou_score: 0.2793 - output_3_f1-score: 0.3462 - output_3_precision: 0.4092 - output_3_recall: 0.6441 - output_2_iou_score: 0.2810 - output_2_f1-score: 0.3477 - output_2_precision: 0.4147 - output_2_recall: 0.6447 - val_loss: 2.7308 - val_output_4_loss: 0.9224 - val_output_3_loss: 0.9047 - val_output_2_loss: 0.9036 - val_output_4_iou_score: 0.5666 - val_output_4_f1-score: 0.5933 - val_output_4_precision: 0.8007 - val_output_4_recall: 0.6785 - val_output_3_iou_score: 0.5494 - val_output_3_f1-score: 0.5818 - val_output_3_precision: 0.7309 - val_output_3_recall: 0.7242 - val_output_2_iou_score: 0.5535 - val_output_2_f1-score: 0.5856 - val_output_2_precision: 0.7371 - val_output_2_recall: 0.7232
Epoch 186/200
118/118 [========

118/118 [==============================] - 106s 897ms/step - loss: 2.5789 - output_4_loss: 0.8804 - output_3_loss: 0.8491 - output_2_loss: 0.8495 - output_4_iou_score: 0.2520 - output_4_f1-score: 0.3119 - output_4_precision: 0.3907 - output_4_recall: 0.5769 - output_3_iou_score: 0.2826 - output_3_f1-score: 0.3515 - output_3_precision: 0.4204 - output_3_recall: 0.6325 - output_2_iou_score: 0.2837 - output_2_f1-score: 0.3522 - output_2_precision: 0.4220 - output_2_recall: 0.6292 - val_loss: 2.7945 - val_output_4_loss: 0.9435 - val_output_3_loss: 0.9253 - val_output_2_loss: 0.9256 - val_output_4_iou_score: 0.5660 - val_output_4_f1-score: 0.5834 - val_output_4_precision: 0.8277 - val_output_4_recall: 0.6567 - val_output_3_iou_score: 0.5641 - val_output_3_f1-score: 0.5903 - val_output_3_precision: 0.7846 - val_output_3_recall: 0.6968 - val_output_2_iou_score: 0.5661 - val_output_2_f1-score: 0.5916 - val_output_2_precision: 0.7909 - val_output_2_recall: 0.6948
Epoch 194/200
118/118 [========

## Draw Model History

In [ ]:
plt.figure(figsize=(30, 20))
# plt.subplot(411)
# plt.plot(history.history['model_4_loss'])
# plt.plot(history.history['val_model_4_loss'])
# plt.title('Model_Prior_5 loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(311)
plt.plot(history.history['output_4_loss'])
plt.plot(history.history['val_output_4_loss'])
plt.title('Model_Prior_4 loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(312)
plt.plot(history.history['output_3_loss'])
plt.plot(history.history['val_output_3_loss'])
plt.title('Model_Prior_3 loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(313)
plt.plot(history.history['output_2_loss'])
plt.plot(history.history['val_output_2_loss'])
plt.title('Model_Prior_2 loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# plt.subplot(311)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')

plt.savefig(HOME_DIR+'/plots/'+'training_loss_IOU_'+MODEL_DESC+'.png')

# Model Evaluation

In [ ]:
""" MOdel Evaluation """
from keras.models import load_model
test_dataset = Dataset(PATH_TO_INPUT,
                       status = 'test',
                classes = Defined_CLASSES,
                augmentation = validation_augmentation(),
                preprocessing = get_preprocessing(preprocess_input),
                
    )

test_dataloader = Dataloader(test_dataset, batch_size=1, shuffle=False)
loading = True
if (loading== True):
    model.load_weights(PATH_TO_SAVE_MODEL+MODEL_DESC+'.h5')


scores = model.evaluate_generator(test_dataloader)
print('__model_4__:\n')
print("model_4_Loss: {:.5}".format(scores[1]))
print("model_4 mean iou_score: {:.5}".format(scores[4]))
print("model_4 mean f1_score: {:.5}".format(scores[5]))
print("model_4 mean precesion: {:.5}".format(scores[6]))
print("model_4 mean recall: {:.5}".format(scores[7]))
                   
print('__model_3__:\n')                  
print("model_3_Loss: {:.5}".format(scores[2]))                   
print("model_3 mean iou_score: {:.5}".format(scores[8]))
print("model_3 mean f1_score: {:.5}".format(scores[9]))
print("model_3 mean precesion: {:.5}".format(scores[10]))
print("model_3 mean recall: {:.5}".format(scores[11]))

print('__model_2__:\n')
print("model_2_Loss: {:.5}".format(scores[3]))
print("model_2 mean iou_score: {:.5}".format(scores[12]))
print("model_2 mean f1_score: {:.5}".format(scores[13]))
print("model_2 mean precesion: {:.5}".format(scores[14]))
print("model_2 mean recall: {:.5}".format(scores[15]))



# print('__model__:\n')
# print("Loss: {:.5}".format(scores[0]))
# print("mean iou_score: {:.5}".format(scores[1]))
# print("mean f1_score: {:.5}".format(scores[2]))

In [ ]:
Save_PATH =HOME_DIR+'Dataset/predictions/'+MODEL_DESC+'/'

for i in range(0,27):
    
    image1, image2, image3,image4,gt_mask, boundary_mask,flight_code = test_dataset[i]
    
    if MODEL_NAME == '_Nine_W_1_1_CONV' or MODEL_NAME == '_Nine_WO_1_1_CONV':
        image = np.concatenate((image1, image2, image3), axis = 2)
        image = np.expand_dims(image, axis=0)
        pr_mask = model.predict(image)
        print(flight_code)
        visualize(
            save_status = False,
            save_path = Save_PATH,
            flight_code = flight_code,
            image1 = image1.squeeze(),
            image2 = image2.squeeze(),
            image3 = image3.squeeze(),
            gt_mask_0=gt_mask[...,0].squeeze(),
            pr_mask_0=pr_mask[...,0].squeeze(),
        ) 
    if MODEL_NAME == '_PRE_LSTM' or MODEL_NAME == '_POST_LSTM':
        image = np.stack((image4, image3,image2), axis = 0)
        image = np.expand_dims(image, axis=0)
        pr_mask4, pr_mask3,pr_mask2 = model.predict(image)
        print(pr_mask3.shape)
        print(flight_code)
        visualize(
            save_status = False,
            save_path = Save_PATH,
            flight_code = flight_code,
            #image5 = image5.squeeze(),
            image4 = image4.squeeze(),
            image3 = image3.squeeze(),
            image2 = image2.squeeze(),
            gt_mask_0=gt_mask[...,0].squeeze(),
            #pr_mask_5=pr_mask5[...].squeeze(),
            pr_mask_4=pr_mask4[...].squeeze(),
            pr_mask_3=pr_mask3[...].squeeze(),
            pr_mask_2=pr_mask2[...].squeeze(),
        )
    else:
        image1 = np.expand_dims(image1, axis=0)
        image2 = np.expand_dims(image2, axis=0)
        image3 = np.expand_dims(image3, axis=0)
        pr_mask3,pr_mask2, pr_mask1 = model.predict([image1,image2,image3])
        print(flight_code)
        visualize(
            save_status = False,
            save_path = Save_PATH,
            flight_code = flight_code,
            image1 = image1.squeeze(),
            image2 = image2.squeeze(),
            image3 = image3.squeeze(),
            #image_0=denormalize(image.squeeze()),
            gt_mask_0=gt_mask[...,0].squeeze(),
            pr_mask1=pr_mask1[...,0].squeeze(),
            pr_mask2=pr_mask2[...,0].squeeze(),
            pr_mask3=pr_mask3[...,0].squeeze(),
        ) 

        
